# Import Libraries


In [226]:
import random
import time
import csv

import numpy as np
import pandas as pd


# Define Classes

In [227]:
class Customer:
    def __init__(self, id, lat, lon, demand):
        self.id = id
        self.lat = lat
        self.lon = lon
        self.demand = demand
        
    def distance(self, customer):
        latDis = abs(self.lat - customer.lat)
        longDis = abs(self.lon - customer.lon)
        distance = 100 * np.sqrt((latDis ** 2) + (longDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.lat) + "," + str(self.lon) + ")"

In [228]:
class Vehicle:
    def __init__(self, type):
        self.type = type
        self.routes = []
        self.rate = None
        self.capacity = None

    def addRoute(self, route):
        self.routes.append(route)

    def setRateCapacity(self):
        if self.type == 'A':
            self.rate = 1.2
            self.capacity = 25
        elif self.type == 'B':
            self.rate = 1.5
            self.capacity = 30
        else:
            self.rate = 0
            self.capacity = 0
    
    def setStartPoint(self):
        self.routes.append(Customer(-1, 4.4184, 114.0932, 0))

In [229]:
class Fitness:
    def __init__(self, vehicle):
        self.vehicle = vehicle
        self.fitness = None
    
    # fitness is based on cost, since distance is directly in correlation with cost,
    # taking cost can be a more accurate representation of the efficiency of the route

    # routes = customer
    def cost(self):
        routes = self.vehicle.routes
        distance = 0

        for i in range (0, len(routes)):
            fromCity = routes[i]
            if i+1 < len(routes):
                toCity = routes[i+1]

            # return to depot
            else:
                toCity = routes[0]
            
            distance += fromCity.distance(toCity)

        return distance*(self.vehicle.rate)
    
    # def fitness(self):
    #     cost = cost(self)

    #     # give higher ranking for lower cost
    #     return abs(1/cost)
            
            

In [230]:
# fill in routes - choose vehicle - ensure routes is selected based on demands


# Generate Population

In [231]:
data = pd.read_csv('data.csv')
data.head()

li = data.values.tolist()

oriRoutes = []
for i in li:
    oriRoutes.append(Customer(i[0], i[1], i[2], i[3]))

solutions = [] #array of vehicles

for s in range(1000):
    print('iteration'+ str(s+1))
    routes = oriRoutes.copy()

    vehicles = []

    while(len(routes) !=0):
        # Randomly choose a vehicle
        type = random.choice(['A', 'B'])
        vehicle = Vehicle(type)
        vehicle.setStartPoint()
        vehicle.setRateCapacity()

        print('Chosen vehicle '+ vehicle.type)

        # Randomly fill in the routes
        filledCapacity = 0

        while(routes):
            selectedRoute = random.choice(routes)
            if((filledCapacity+selectedRoute.demand) <= vehicle.capacity):
                filledCapacity += selectedRoute.demand
                vehicle.addRoute(selectedRoute)
                routes.remove(selectedRoute)
                print('Route ' + str(selectedRoute.id) + ', demand ' + str(selectedRoute.demand))
            else:
                break
            

        vehicles.append(vehicle)

    solutions.append(vehicles)
    
solutions


'iteration1'
'Chosen vehicle A'
'Route 7.0, demand 3.0'
'Route 4.0, demand 6.0'
'Route 6.0, demand 8.0'
'Route 1.0, demand 5.0'
'Chosen vehicle A'
'Route 8.0, demand 6.0'
'Route 10.0, demand 8.0'
'Route 5.0, demand 5.0'
'Route 9.0, demand 5.0'
'Chosen vehicle A'
'Route 2.0, demand 8.0'
'Route 3.0, demand 3.0'
'iteration2'
'Chosen vehicle A'
'Route 10.0, demand 8.0'
'Route 3.0, demand 3.0'
'Route 7.0, demand 3.0'
'Route 2.0, demand 8.0'
'Chosen vehicle B'
'Route 5.0, demand 5.0'
'Route 8.0, demand 6.0'
'Route 1.0, demand 5.0'
'Route 6.0, demand 8.0'
'Route 9.0, demand 5.0'
'Chosen vehicle A'
'Route 4.0, demand 6.0'
'iteration3'
'Chosen vehicle A'
'Route 2.0, demand 8.0'
'Route 7.0, demand 3.0'
'Route 4.0, demand 6.0'
'Route 3.0, demand 3.0'
'Route 1.0, demand 5.0'
'Chosen vehicle B'
'Route 10.0, demand 8.0'
'Route 9.0, demand 5.0'
'Route 8.0, demand 6.0'
'Route 5.0, demand 5.0'
'Chosen vehicle A'
'Route 6.0, demand 8.0'
'iteration4'
'Chosen vehicle A'
'Route 4.0, demand 6.0'
'Route 6.0,

[[<__main__.Vehicle at 0x22cd1ea9df0>,
 [<__main__.Vehicle at 0x22cd1eb2090>, <__main__.Vehicle at 0x22cd1eb25d0>],
 [<__main__.Vehicle at 0x22cd1eb1c10>, <__main__.Vehicle at 0x22cd1eb1b50>],
 [<__main__.Vehicle at 0x22cd1eb3410>, <__main__.Vehicle at 0x22cd1eb2f00>],
 [<__main__.Vehicle at 0x22cd1eb3e60>, <__main__.Vehicle at 0x22cd1eb3da0>],
 [<__main__.Vehicle at 0x22cd1f003e0>, <__main__.Vehicle at 0x22cd1f004a0>],
 [<__main__.Vehicle at 0x22cd1f007a0>, <__main__.Vehicle at 0x22cd1f00860>],
 [<__main__.Vehicle at 0x22cd1f01460>, <__main__.Vehicle at 0x22cd1f01520>],
 [<__main__.Vehicle at 0x22cd1f01a60>, <__main__.Vehicle at 0x22cd1f01b20>],
 [<__main__.Vehicle at 0x22cd1f02060>, <__main__.Vehicle at 0x22cd1f02120>],
 [<__main__.Vehicle at 0x22cd1f021e0>, <__main__.Vehicle at 0x22cd1f022a0>],
 [<__main__.Vehicle at 0x22cd1f03320>, <__main__.Vehicle at 0x22cd1f033e0>],
 [<__main__.Vehicle at 0x22cd1f04260>, <__main__.Vehicle at 0x22cd1f04320>],
 [<__main__.Vehicle at 0x22cd1f043e0>

In [232]:
# routes do not need to include depot at end, but shoudl include depot at start

# Tournament Selection

In [233]:
def fitness(solution):
    solutionCost = 0
    for vehicle in solution:
        fitness = Fitness(vehicle)
        cost = fitness.cost()
        solutionCost += cost
    return solutionCost

# Randomly choose 20 samples, get one parent
def getParent():
    choices = random.sample(solutions, 20)

    chosenSolutionCost = []
    for vehicles in choices:
        solutionCost = fitness(vehicles)
        chosenSolutionCost.append(solutionCost)


    minpos = chosenSolutionCost.index(min(chosenSolutionCost))

    return choices[minpos]

firstParent = getParent()
secondParent = getParent()

print(firstParent)
secondParent


# Crossover

In [234]:
def findMissing(li):
    return sorted(set(range(1, 11)).difference(li))


def convertToOneDList(li):
    convertedList = []
    for i in range(len(li)):
        for j in range(len(li[i])):
            convertedList.append(int(li[i][j]))
    return convertedList

def getDuplicatedElements(li):
    duplicates = []
    for x in li:
        if x not in duplicates and li.count(x) >1:
            duplicates.append(x)
        
    return duplicates


In [235]:
# drop start point when crossing over

# first = [[3.0, 4.0, 2.0, 6.0], [7.0, 1.0, 5.0, 9.0], [8.0, 10.0]]
# firstVehicles = ['B', 'A', 'B']
# second = [[4.0, 6.0, 5.0], [3.0, 1.0, 2.0, 7.0, 10.0], [8.0, 9.0]]
# secondVehicles = ['A', 'B', 'A']


# selectedVehicleIdx = 1
# selectedToSwapVehicle = [3.0, 1.0, 2.0, 7.0, 10.0]
# selectedToSwapVehicle2 = [7.0, 1.0, 5.0, 9.0]



def getCrossoverOffspring(firstParent, secondParent):
    childVehicles = []

    first = [[],[],[]]
    firstVehicles = []

    for i in range(len(firstParent)):
        #some solutions only have 2 vehicles
        if (firstParent[i].routes):
            firstParent[i].routes.pop(0)
        firstVehicles.append(firstParent[i].type)
        # for every route
        for j in range(len(firstParent[i].routes)):
            first[i].append(firstParent[i].routes[j].id)



    second = [[],[],[]]
    secondVehicles = []


    for i in range(len(secondParent)):
        if (secondParent[i].routes):
            secondParent[i].routes.pop(0)
        secondVehicles.append(secondParent[i].type)
        # for every route
        for j in range(len(secondParent[i].routes)):
            second[i].append(secondParent[i].routes[j].id)


    # Random Choice of 1,2,3,4
    choice = random.randint(1, 4)
    # Case 1 - first parent is  main, two vehicles remain, one vehicle changes
    if (choice == 1 or choice == 2):
        if (choice == 1):
            parent = first.copy()
            secondary = second.copy()
            selectedVehicleIdx = random.randint(0, len(parent)-1)
            selectedToSwapVehicle = random.choice(secondary)
            parent[selectedVehicleIdx] = selectedToSwapVehicle
            parent = convertToOneDList(parent)
            childVehicles = firstVehicles.copy()

        # Case 2 - second parent is main, two vehicles remain, one vehicle changes
        elif (choice == 2):
            parent = second.copy()
            secondary = first.copy()
            selectedVehicleIdx = random.randint(0, len(parent)-1)
            selectedToSwapVehicle = random.choice(secondary)
            parent[selectedVehicleIdx] = selectedToSwapVehicle
            parent = convertToOneDList(parent)
            childVehicles = secondVehicles.copy()

        missing = findMissing(parent)
        duplicates = getDuplicatedElements(parent)

        child = []
        for i in parent:
            if i not in child:
                child.append(i)
            else:
                if(missing):
                    child.append(missing[0])
                    missing.pop(0)

        if (missing):
            child += missing


    elif (choice == 3 or choice == 4):
        # Case 3 - take first parent as a whole, no crossover, direcly sent to mutation
        if (choice == 3):
            child = convertToOneDList(first)
            childVehicles = firstVehicles.copy()
        # Case 4 - take second parent as a whole, no crossover, direcly sent to mutation
        elif (choice == 4):
            child = convertToOneDList(second)
            childVehicles = secondVehicles.copy()

    return child, childVehicles


# Mutation

In [236]:
# Vehicle Mutation
def vehicleMutation(li):
    idx = random.randint(0, len(li)-1)
    if li[idx] == 'A':
        li[idx] = 'B'
    else:
        li[idx] = 'A'
    
    return li

# Routes Mutation - Swap
def routeMutation(li):
    if(len(li) >= 1):
        aIdx = random.randint(0, len(li)-1)
        bIdx = random.randint(0, len(li)-1)

        temp = li[aIdx]
        li[aIdx] = li[bIdx]
        li[bIdx] = temp

    return li

In [237]:
# print(childVehicles)
# print(vehicleMutation(childVehicles))

# print(child)
# print(routeMutation(child))

def getMutatedOffspring(first, second):

    child, childVehicles = getCrossoverOffspring(first, second)
    mutatedChild = routeMutation(child)
    mutatedVehicles = vehicleMutation(childVehicles)

    return mutatedChild, mutatedVehicles




# Convert back to class

In [238]:
# convert routes to Customer class
data = pd.read_csv('data.csv')
data.head()

li = data.values.tolist()
print(li)

routes = []
for i in li:
    routes.append(Customer(i[0], i[1], i[2], i[3]))


[[1.0, 4.3555, 113.9777, 5.0],
 [2.0, 4.3976, 114.0049, 8.0],
 [3.0, 4.3163, 114.0764, 3.0],
 [4.0, 4.3184, 113.9932, 6.0],
 [5.0, 4.4024, 113.9896, 5.0],
 [6.0, 4.4142, 114.0127, 8.0],
 [7.0, 4.4804, 114.0734, 3.0],
 [8.0, 4.3818, 114.2034, 6.0],
 [9.0, 4.4935, 114.1828, 5.0],
 [10.0, 4.4932, 114.1322, 8.0]]


In [239]:
def getOffspring(first, second):
    mutatedChild, mutatedVehicles = getMutatedOffspring(first, second)
    print(mutatedChild)
    print(mutatedVehicles)

    offspring = []
    for i in mutatedVehicles:
        vehicle = Vehicle(i)
        vehicle.setStartPoint()
        vehicle.setRateCapacity()

        filledCapacity = 0

        while(mutatedChild):
            idx = mutatedChild[0]
            selectedRoute = routes[idx-1]
            if((filledCapacity+selectedRoute.demand) <= vehicle.capacity):
                filledCapacity += selectedRoute.demand
                vehicle.addRoute(selectedRoute)
                mutatedChild.pop(0)
            else:
                break
        
        offspring.append(vehicle)

    return offspring
    

# offspring = getOffspring()
# fitnessCost = fitness(offspring)

print(fitnessCost)

165.65358233238467


In [240]:
# Generate two offspring
first = getParent()
second = getParent()

for i in range(1000):
    offspring1 = getOffspring(first, second)
    fitness1 = fitness(offspring1)

    offspring2 = getOffspring(first, second)
    fitness2 = fitness(offspring2)

    averageScore = (fitness1+fitness2)/2

    print("Iteration " + str(i+1))
    print('Score: ' + str(averageScore))

[7, 10, 4, 5, 9, 1, 3, 6, 8, 2]
['A', 'B', 'B']
[7, 5, 8, 3, 4, 10, 9]
['A', 'A', 'B']
'Iteration 1'
'Score: 173.93349916482498'
[4, 5, 9, 8, 3, 1, 2, 6, 7, 10]
['A', 'A', 'A']
[5, 6]
['B', 'A', 'B']
'Iteration 2'
'Score: 95.21208314227884'
[]
['A', 'A', 'A']
[10, 2, 3, 4, 5, 6, 7, 8, 9, 1]
['A', 'A', 'A']
'Iteration 3'
'Score: 94.3181690223977'
[4, 2, 3, 1, 5, 6, 7, 8, 9, 10]
['A', 'B', 'B']
[]
['A', 'B', 'A']
'Iteration 4'
'Score: 92.76944741779819'
[8, 2, 3, 4, 5, 6, 7, 1, 9, 10]
['B', 'A', 'A']
[1, 3, 2, 4, 5, 6, 7, 8, 9, 10]
['B', 'A', 'A']
'Iteration 5'
'Score: 193.89856420867528'
[1, 2, 3, 4, 5, 9, 7, 8, 6, 10]
['A', 'A', 'A']
[]
['A', 'A', 'A']
'Iteration 6'
'Score: 91.83993550001368'
[]
['B', 'A', 'B']
[1, 2, 3, 4, 5, 6, 9, 8, 7, 10]
['A', 'A', 'A']
'Iteration 7'
'Score: 76.3392116589693'
[1, 2, 3, 4, 10, 6, 7, 8, 9, 5]
['A', 'A', 'A']
[1, 2, 3, 4, 6, 5, 7, 8, 9, 10]
['B', 'A', 'A']
'Iteration 8'
'Score: 181.06006768342058'
[1, 9, 3, 4, 5, 6, 7, 8, 2, 10]
['A', 'A', 'A']
[1, 6